In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [ ]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_risk_df = pd.read_csv("lending_data.csv")

# Review the DataFrame
lending_risk_df.sample(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
55801,7600.0,6.339,40200,0.253731,2,0,10200,0
24988,10900.0,7.771,53700,0.441341,5,1,23700,0
42603,11100.0,7.834,54300,0.447514,5,1,24300,0
53159,9400.0,7.131,47700,0.371069,3,0,17700,0
61031,9200.0,7.031,46800,0.358974,3,0,16800,0
26717,10500.0,7.573,51900,0.421965,4,1,21900,0
38652,19900.0,11.596,89700,0.665552,13,2,59700,0
46584,7900.0,6.483,41600,0.278846,2,0,11600,0
47699,10400.0,7.563,51800,0.420849,4,1,21800,0
28073,6600.0,5.930,36400,0.175824,1,0,6400,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [ ]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_risk_df['loan_status']

# Separate the X variable, the features
x = lending_risk_df.drop(columns= 'loan_status')

In [ ]:
# Review the y variable Series
y.sample(10)

7421     0
8347     0
40136    0
59726    0
57139    0
17297    0
74006    0
28163    0
48806    0
5460     0
Name: loan_status, dtype: int64

In [ ]:
# Review the X variable DataFrame
x.sample(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
38783,12500.0,8.438,60000,0.500000,6,1,30000
57350,8500.0,6.735,44000,0.318182,3,0,14000
20369,9800.0,7.306,49400,0.392713,4,0,19400
39900,11000.0,7.801,54000,0.444444,5,1,24000
31430,10100.0,7.407,50300,0.403579,4,1,20300
66935,10200.0,7.459,50800,0.409449,4,1,20800
31470,8200.0,6.589,42600,0.295775,2,0,12600
12733,9300.0,7.085,47300,0.365751,3,0,17300
37656,8400.0,6.702,43700,0.313501,3,0,13700
66790,10200.0,7.442,50600,0.407115,4,1,20600


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [ ]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [ ]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1, stratify= y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [ ]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model_1 = LogisticRegression(random_state = 1)

# Fit the model using training data
model_1.fit(x_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [ ]:
# Make a prediction using the testing data
y_pred = model_1.predict(x_test)

In [ ]:
# Show the resulted data side by side
outcome = pd.DataFrame({'prediction': y_pred, 'real': y_test}).reset_index(drop = True)
outcome.head()

,prediction,real
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, y_pred)

0.9442676901753825

In [ ]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_pred, y_test))

[[18679    67]
 [   80   558]]


In [ ]:
# Print the classification report for the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model, trained on the original data, predicts the two labels with an accuracy of 94.4%. Given that the f1-score is 1.00, the model is quite effective at forecasting the healthy loans. The algorithm might, however, do a better job at predicting high-risk loans, as it only predicted 87% of all actual high-risk loans and 89% of all high-risk loans in the dataset correctly.

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [ ]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler_model = RandomOverSampler(random_state = 1)

# Fit the original training data to the random_oversampler model
x_resample, y_resample = random_oversampler_model.fit_resample(x_train, y_train)

In [ ]:
# Count the distinct values of the resampled labels data
y_resample.value_counts()

0    56277
1    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [ ]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model_2 = LogisticRegression(random_state = 1)

# Fit the model using the resampled training data
model_2.fit(x_resample, y_resample)

# Make a prediction using the testing data
y_pred_1 = model_2.predict(x_test)

In [ ]:
outcome_1 = pd.DataFrame({"Prediction": y_pred_1, "Actual": y_test}).reset_index(drop=True)
outcome_1.head()

,Prediction,Actual
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, y_pred_1)

0.9959744975744975

In [ ]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, y_pred_1))

[[18668    91]
 [    2   623]]


In [ ]:
# Print the classification report for the model
print(classification_report(y_test, y_pred_1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model trained using oversampled data performs better than the model trained on the original data in terms of accuracy in predicting the two labels and recall in accurately identifying the high-risk loans in the dataset.